# Import Necessary Libraries

In [ ]:
import numpy as np
from PIL import Image

# for joining files paths for tif images
import os

# for removing the background
import cv2
from cv2_rolling_ball import subtract_background_rolling_ball

# Normalizing all Images in Batches

The 100% full cryo images were normalized as:
1) Rolling Ball 200
2) CLAHE: clipsize= 2.0, tilesize= 8.0

The 100% empty cryo images were normalized as:
1) CLAHE: clipsize= 2.0, tilesize= 8.0
2) Rolling Ball 200
3) CLAHE: clipsize= 2.0, tilesize= 100

### Specify File Paths and Batch Size

In [ ]:
####################################### Load the Data ########################################

img_path = "your_path_here"                                                                                                                                    # change
save_CLAHE_path = "your_path_here"                                                                                                                             # change
full_empty = "name_of_your_data_here"                                                                                                                          # change

idx_last_batch = 0                                                                                                                                             # change
num_images_this_batch = 130                                                                                                                                    # change
idx_end_this_batch = idx_last_batch + num_images_this_batch + 1
idx_end_this_batch

### Normalizing the Images

In [ ]:
# Convert Tiffs to RGB Images with 3 channels
# Note: These images have been resized to contain 3 channels they have also been converted 

######################## Resize the images to 3 channels ########################################

resized_images = []

print("Resizing the images to 3 channels...")
# loop over all the images
# use the next 130 images after 250 
for i, file in enumerate(os.listdir(img_path)[idx_last_batch:idx_end_this_batch]):

    # Construct the full path to the image
    
    # only read in the tif files
    if not file.endswith(".tif"):
        # if continue then the for loop will go to the next iteration
        continue
    
    img_full_path = os.path.join(img_path, file)

    # Open the image and convert it to an array
    img_rgb = Image.open(img_full_path)
    img_rgb = np.array(img_rgb).astype(np.uint8)

    # Duplicate the first channel into 3 channels
    img_rgb = np.repeat(img_rgb[:, :, np.newaxis], 3, axis=2)
    
    # save the image in the list
    resized_images.append(img_rgb)
    
print("Applying CLAHE to the images...")
# Process each image from the list and save each image as you go
for i, img_rgb in enumerate(resized_images):
    # Define the image
    # Convert the image to LAB
    lab_img = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_img)
    l = np.array(l, dtype=np.uint8)

    ############################# Apply CLAHE once to the image ########################################
    final_img = l

    # Apply CLAHE to the image
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe1_img = clahe.apply(final_img)
    
    ############################# Apply Rolling Ball to the image ########################################
    radius = 200
    rolling_ball_img, background = subtract_background_rolling_ball(clahe1_img, radius, light_background=True, use_paraboloid=False,
                                                             do_presmooth=True)
    
    ############################# Apply CLAHE a second time to the image ########################################
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(100, 100))
    clahe2_img = clahe.apply(rolling_ball_img)

    # Convert the image to RGB    
    clahe_img_rgb = cv2.cvtColor(clahe2_img, cv2.COLOR_GRAY2RGB)
    
    # Save only the first channel
    clahe_img_rgb_1c = clahe_img_rgb[:, :, 0]

    # Save the image as a tif file
    image_save_CLAHE_path = os.path.join(save_CLAHE_path, "Cryo_CLAHE_" + full_empty + str(i+idx_last_batch+1) + ".tif")
    cv2.imwrite(image_save_CLAHE_path, clahe_img_rgb_1c)
